In [1]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import imageio

#### EVA

In [2]:
import glob
eva=[]
for image_path in glob.glob("image_aumentation/Eva/*.jpg"):
    image = imageio.imread(image_path)
    eva.append(image)

In [5]:
eva_res=[]
width = 224
height = 224
for image in eva:
    image_resized = cv2.resize(image,(width,height),interpolation = cv2.INTER_CUBIC)
    eva_res.append(image_resized)

In [7]:
edited_eva=[]
for i in range(len(eva_res)):
    color_eva= cv2.cvtColor(eva_res[i],cv2.COLOR_BGR2RGB)
    edited_eva.append(color_eva)

In [3]:
len(eva)

184

#### EDGAR

In [4]:
import glob
edgar=[]
for image_path in glob.glob("image_aumentation/Edgar/*.jpg"):
    image = imageio.imread(image_path)
    edgar.append(image)

In [6]:
edgar_res=[]
width = 224
height = 224
for image in edgar:
    image_resized = cv2.resize(image,(width,height),interpolation = cv2.INTER_CUBIC)
    edgar_res.append(image_resized)

In [8]:
edited_edgar=[]
for i in range(len(eva_res)):
    color_edgar= cv2.cvtColor(edgar_res[i],cv2.COLOR_BGR2RGB)
    edited_edgar.append(color_edgar)

### Pre-pocesat imatges

In [9]:
eva = np.asarray(edited_eva)

In [10]:
edgar = np.asarray(edited_edgar)

### Train/test

In [11]:
total = eva.shape[0]

In [12]:
train = total*0.8
train

147.20000000000002

#### EVA

In [13]:
eva = np.split(eva,[147])

In [15]:
eva_train = eva[0]
eva_test = eva[1]

In [17]:
label_eva_train=np.zeros((147))
label_eva_test=np.zeros((37))
#We define that if we have a 0 is a Eva

In [19]:
eva_train.shape

(147, 224, 224, 3)

#### EDGAR

In [14]:
edgar = np.split(edgar,[147])


In [16]:
edgar_train = edgar[0]
edgar_test = edgar[1]

In [18]:
label_edgar_train=np.ones((147))
label_edgar_test=np.ones((37))

In [20]:
edgar_train.shape

(147, 224, 224, 3)

#### BOTH

In [21]:
train_x=np.concatenate((eva_train,edgar_train))
test_x=np.concatenate((eva_test,edgar_test))
train_y=np.concatenate((label_eva_train,label_edgar_train))
test_y=np.concatenate((label_eva_test,label_edgar_test))

## MODEL

In [22]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten,BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

#from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

import h5py  #libreria de los gatos
import scipy
from PIL import Image
from scipy import ndimage

Using TensorFlow backend.


In [23]:
import tensorflow as tf
from tensorflow import keras
# CNN parameters
batch_size = 32
num_classes = 1
epochs = 20

In [24]:
# setup model
model = Sequential()
#conv 1
model.add(Conv2D(16, kernel_size=(3, 3),
                activation='relu',
                input_shape=(224,224,3)))
model.add(BatchNormalization(axis=3))
#max pooling 1
model.add(MaxPooling2D(pool_size=(2,2), strides=2)) #Strides


#conv 2
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(BatchNormalization(axis=3))
#max pooling 2
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
#model.add(Dropout(0.25))


#conv 3
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization(axis=3))
#max pooling 3
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))


#conv 4
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(BatchNormalization(axis=3))
#max pooling 3
model.add(MaxPooling2D(pool_size=(2, 2), strides=2))


#flatten
model.add(Flatten())
# fc layer 1
model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.5))
# fc layer 2
model.add(Dense(1, activation='sigmoid'))

W0829 17:10:47.095624 139908172986176 deprecation_wrapper.py:119] From /home/eva/anaconda3/envs/akademyai/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0829 17:10:47.142026 139908172986176 deprecation_wrapper.py:119] From /home/eva/anaconda3/envs/akademyai/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0829 17:10:47.160050 139908172986176 deprecation_wrapper.py:119] From /home/eva/anaconda3/envs/akademyai/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0829 17:10:47.255405 139908172986176 deprecation_wrapper.py:119] From /home/eva/anaconda3/envs/akademyai/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is de

In [25]:
# compile model
#adam=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)
keras.optimizers.Adam(lr=0.001)

adamax=keras.optimizers.Adamax(lr=0.2, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
adagrad=keras.optimizers.Adagrad(lr=0.001, epsilon=None, decay=0.0)

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

W0829 17:10:48.053052 139908172986176 deprecation_wrapper.py:119] From /home/eva/anaconda3/envs/akademyai/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0829 17:10:48.073872 139908172986176 deprecation.py:323] From /home/eva/anaconda3/envs/akademyai/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 16)      448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 222, 222, 16)      64        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 16)      2320      
_________________________________________________________________
batch_normalization_2 (Batch (None, 109, 109, 16)      64        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 52, 52, 32)        4640      
__________

## AQUI ES DONDE ENTRENAMOS EL MODELO DEFINIDO ANTERIORMENTE CON NUESTROS DATOS

In [26]:
model.fit(train_x, train_y, epochs=13)

Epoch 1/13
294/294 [==============================] - 13s 45ms/step - loss: 0.8809 - acc: 0.8129
Epoch 2/13
294/294 [==============================] - 9s 31ms/step - loss: 0.1910 - acc: 0.9660
Epoch 3/13
294/294 [==============================] - 10s 35ms/step - loss: 0.0805 - acc: 0.9864
Epoch 4/13
294/294 [==============================] - 10s 34ms/step - loss: 0.0752 - acc: 0.9796
Epoch 5/13
294/294 [==============================] - 10s 34ms/step - loss: 5.2462e-04 - acc: 1.0000
Epoch 6/13
294/294 [==============================] - 10s 33ms/step - loss: 0.0047 - acc: 0.9966
Epoch 7/13
294/294 [==============================] - 10s 33ms/step - loss: 0.0011 - acc: 1.0000
Epoch 8/13
294/294 [==============================] - 10s 35ms/step - loss: 1.2564e-04 - acc: 1.0000
Epoch 9/13
294/294 [==============================] - 9s 32ms/step - loss: 2.7293e-05 - acc: 1.0000
Epoch 10/13
294/294 [==============================] - 9s 30ms/step - loss: 1.3340e-05 - acc: 1.0000
Epoch 11/13
294/

In [27]:
# evaluate
test_loss, test_acc = model.evaluate(test_x, test_y)
print('test accuracy: ', test_acc)

74/74 [==============================] - 1s 17ms/step
test accuracy:  0.9594594594594594


In [28]:
# evaluate model
score = model.evaluate(test_x, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.3306262898997991
Test accuracy: 0.9594594594594594


In [29]:
predictions = model.predict(test_x)

In [30]:
predictions

array([[0.0000000e+00],
       [0.0000000e+00],
       [5.1605499e-01],
       [0.0000000e+00],
       [0.0000000e+00],
       [1.4388561e-04],
       [0.0000000e+00],
       [5.0824881e-04],
       [4.4703484e-05],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [1.0921997e-01],
       [0.0000000e+00],
       [9.9953884e-01],
       [0.0000000e+00],
       [4.0769577e-05],
       [0.0000000e+00],
       [0.0000000e+00],
       [1.0043383e-05],
       [0.0000000e+00],
       [0.0000000e+00],
       [1.3709068e-06],
       [0.0000000e+00],
       [5.9604645e-08],
       [0.0000000e+00],
       [0.0000000e+00],
       [9.9999994e-01],
       [4.4703484e-07],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [0.0000000e+00],
       [9.9999982e-01],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.0000000e+00],
       [1.000000

In [ ]:
print('x_test shape: ', test_x.shape)
x_test_plt = np.squeeze(test_x)
print('x_test_plt shape: ', x_test_plt.shape)
print('predictions: ', predictions.shape)

In [ ]:
 #see results
i = 26
plt.imshow(x_test_plt[i])
print('Prediction: ' , np.where(predictions[i] <=0.5,'Eva','Edgar'))
print('With an accuracy of:', np.round(score[1]*100,2), '%' )

### SAVE THE MODEL 

In [52]:
from sklearn.externals import joblib 

/home/eva/anaconda3/envs/akademyai/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [53]:
joblib.dump(model, 'cnn_model.pkl')

['cnn_model.pkl']